# Code to web scrape Indeed UK

In [ ]:
import pandas as pd

## Function that does the scraping

In [512]:
def extract_title(soup): 
    #initialse
    jobs = []
    company=[]
    pay=[]
    descr=[]
    
    #"job_seen_beacon" is the div for each job panel 
          # would be better to use <a> tag with 
    for div in soup.find_all(name="div", attrs={"class":"job_seen_beacon"}):

        #for the company name is in span tag with class=companyName
        aComp="-"
        for a in div.find_all(name="span", attrs={"class":"companyName"}):
            try:
                aComp=a.string
            except:
                print('exception comp')
                pass 
        company.append(a.string)
        
        aJob='-'
        for ab in div.find_all(name="span"):
            try:
                aJob=ab['title']              
            except: #Exception
                pass
        jobs.append(aJob)
        
        aPay="0"
        for abc in div.find_all(name="span", attrs={"class":"salary-snippet"}):
            try:
                aPay=abc.string
            except:
                print('exception pay')
                pass
        pay.append(aPay)

        
        aDescr=''
        abc = div.find_all(name='div',attrs={"class":"job-snippet"})
        for abcd in abc[0].find_all(name='li'):
            try:
                aDescr=aDescr+abcd.getText()+'-'
            except:
                print('exception decr')
                pass
        
        descr.append(aDescr)
        
    sa=soup.find_all(name='a',attrs={"target":"_blank"})
    urlss=[]
    
    strURLstart='https://uk.indeed.com/viewjob?'
    for aSect in sa:
        li=[]
        try:
            aCheck=aSect['id']
            li.append(aCheck)
            urlss.append(strURLstart+aSect['href'])
        except:
            pass
            
    return jobs, company, pay, descr, urlss


###################################################################################################
###################################################################################################


## Cycle through pages and call inner function 'extract_title'
def extract_outer(jobtype,location,max_results):
    import requests
    import bs4
    from bs4 import BeautifulSoup
    import pandas as pd
    import numpy as np
    import copy as copy
    
    #Split job into two string with plus - i.e. "road sweeper"->"road + sweeper"
    jobtype=jobtype.replace(' ',' + ')
    
    #initialise values
    df=pd.DataFrame(columns=['Company','Job Title','Pay','Details','URL'])
    
    #Scroll through pages i.e. 0-9 in page1 --10-19 in page2 (in theory!)
    i=0
    for start in range(0, max_results, 10):#Add 10 on till a max
        
        #initialise values
        num = (len(df) + 1)
        jobs_, company_, pay_, descr_, urlss_ = [], [] , [], [], []
        num=str(start)
        
        #get URL
        URL="https://uk.indeed.com/jobs?q="+jobtype+"&l="+location+"&start="+num#conducting a request of the stated URL above:
        page = requests.get(URL)#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
        soup = BeautifulSoup(page.text, "html.parser")#printing soup in a more structured tree format that makes for easier reading
        
        #call the inner function
        jobs_, company_, pay_, descr_, urlss_=extract_title(soup)
        
        # append the lists
        if i==0:          ## on 1st go assign df values
            df['Company']=company_
            df['Job Title']=jobs_
            df['Pay']=pay_
            df['Details']=descr_
            df['URL']=urlss_
        else:            ## on next ones concat df with a new df 
            df_=pd.DataFrame(columns=['Company','Job Title','Pay','Details','URL'])
            df_['Company']=company_
            df_['Job Title']=jobs_
            df_['Pay']=pay_
            df_['Details']=descr_
            df_['URL']=urlss_
            
            df=pd.concat(
                [df,df_ 
                
                ]
                        )
            del df_
        i=i+1
    
        df=df.reset_index(drop=True)#reset index- drop stops index becoming a column
    return df

In [511]:
jobtitle='call centre'
maxRes=50
locat="Swansea%2C+Swansea"
fname="indeed_"+jobtitle+'_'+locat+'_maxRes='+str(maxRes)+'.csv'
df=extract_outer(jobtitle,locat,maxRes)
df

,Company,Job Title,Pay,Details,URL
0,Sitel,Customer Service Representative / Sales Repres...,0,Our call centre agents take inbound calls and ...,https://uk.indeed.com/viewjob?/pagead/clk?mo=r...
1,Countrywide Hearing,Call Centre Claims Handler,"£21,000 - £24,000 a year",Diarise calls and contact to ensure all enquir...,https://uk.indeed.com/viewjob?/pagead/clk?mo=r...
2,Unite Wills,Funeral Planner – Call Centre Agent,"£18,000 - £22,000 a year",Proven track record in call centres with a str...,https://uk.indeed.com/viewjob?/pagead/clk?mo=r...
3,The Consumer Helpline,Outbound Telesales Agent,£360 a week,Provide excellent customer service during sale...,https://uk.indeed.com/viewjob?/pagead/clk?mo=r...
4,iQuote Online,Call Centre Agent - Life Insurance,"£350 - £1,000 a week",Keep records of calls and sales and note usefu...,https://uk.indeed.com/viewjob?/pagead/clk?mo=r...
...,...,...,...,...,...
60,Dunraven Windows,Telesales Canvasser,"£18,137 a year",Established in 1979 Dunraven Windows have been...,https://uk.indeed.com/viewjob?/company/Dunrave...
61,MVF CAREERS,Telemarketing Consultant,0,As a Telemarketing Consultant you will be call...,https://uk.indeed.com/viewjob?/rc/clk?jk=f5518...
62,Page Personnel - UK,Telesales / Phone Sales (B2B) - Permanent Work...,"£16,000 - £23,000 a year",3 to 3.5 hours of connected calls.-Undertake s...,https://uk.indeed.com/viewjob?/rc/clk?jk=bf2be...
63,Jomo People Ltd,Telesales Executive,"£19,200 a year",Proven track record of working within a contac...,https://uk.indeed.com/viewjob?/company/Jomo-Pe...


## And Save if you want

In [ ]:

df.to_csv("my ".csv”, encoding=’utf-8')